In [25]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from textwrap import shorten
import requests

from bs4 import BeautifulSoup

In [26]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [27]:
df = pd.read_csv('19 hotels Ashford REIT expects to transfer to lender - Sheet1.csv')

In [28]:
df['geo_address'] = df['full_address']

In [29]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [30]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Image URL Collector

In [31]:
# df['image_links'] = ''
# links = []
# for url in df['story_link']:
#     try:
#         page = requests.get(url)
#         soup = BeautifulSoup(page.content, 'html.parser')
#         images = soup.find('img',{'class':'MainArtImage_image__2YtGm'})
#         images = images['src']
#         links.append(images)
#     except:
#         links.append('no_image')

# df['image_links'] = links

In [32]:
# print(df['story_link'].iloc[0])

In [33]:
# df.at[20,'image_img']=('<img src="Aimco.jpg" width="256" height="156">')
# df['image_links'].iloc[1]

In [34]:
# df.at[4,'image_links']=('https://therealdeal.com/wp-content/uploads/2019/10/trd-logo-16x9.jpg')

In [35]:
# df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
# df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
# df['remaining_desc']

# df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
# df['description_link']

In [36]:
df.columns

Index(['full_address', 'Name of hotel', 'description', 'story_link',
       'geo_address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [37]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    hotel_name = df['Name of hotel'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(address) + '''<br><br>
    <strong>Name: </strong>{}'''.format(hotel_name) + '''<br>
    </html>
    '''
    return html

In [41]:
import branca

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=4)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Hotels Ashford Hospitality Trust expects to transfer to lender')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True, width=250))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [42]:
m.save('index.html')

## Map URL Snagger

In [43]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/hotels_Ashford_REIT_transfer_to_lender
